In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [50]:
image_size = 227
num_epochs = 10
batch_size = 64

device = ('cuda' if torch.cuda.is_available() else 'cpu')
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print(torch.__version__)
print(device)

1.13.1+cu116
cuda


In [51]:
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor()
])

In [52]:
train_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = transform
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = transform
)

In [53]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

In [54]:
next(iter(train_loader))[0].shape

torch.Size([64, 1, 227, 227])

In [55]:
class AlexNetFashionMNIST(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=96, kernel_size=11, stride=4, padding=0),
            #input_size : 1x227x227
            #kernel_number : 96
            #output_w : ((227-11)/4) + 1) = 55
            #output_size : 96x55x55
            nn.ReLU(), # size 동일
            nn.MaxPool2d(3,2)
            #((55-3)/2)+1) = 27
            #최종 96x27x27 feature map 생성 
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2),
            #256x27x27
            nn.ReLU(),#256x27x27
            nn.MaxPool2d(3, 2)
            #256x13x13
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1),
            #13 유지
            nn.ReLU()
            #384x13x13
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
            #13 유지
            nn.ReLU()
            #384x13x13
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)# 13 -> 6
            #256x6x6
        )
        
        
        self.fc1 = nn.Linear(in_features=256*6*6, out_features=4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096,10)
    
    
    def forward(self,x): #input_size = 64x1x227x227
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out)
        out = out.view(out.size(0),-1)# 64x(256*6*6)
        
        out = F.relu(self.fc1(out))
        out = F.dropout(out, 0.5)
        out = F.relu(self.fc2(out))
        out = F.dropout(out, 0.5)
        out = self.fc3(out)
        out = F.log_softmax(out, dim=1)
        
        return out

In [56]:
model = AlexNetFashionMNIST().to(device)
criterion = F.nll_loss
optimizer = optim.Adam(model.parameters())

In [57]:
from torchsummary import summary as s

s(model, (1,227,227), batch_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [64, 96, 55, 55]          11,712
              ReLU-2           [64, 96, 55, 55]               0
         MaxPool2d-3           [64, 96, 27, 27]               0
            Conv2d-4          [64, 256, 27, 27]         614,656
              ReLU-5          [64, 256, 27, 27]               0
         MaxPool2d-6          [64, 256, 13, 13]               0
            Conv2d-7          [64, 384, 13, 13]         885,120
              ReLU-8          [64, 384, 13, 13]               0
            Conv2d-9          [64, 384, 13, 13]       1,327,488
             ReLU-10          [64, 384, 13, 13]               0
           Conv2d-11          [64, 256, 13, 13]         884,992
             ReLU-12          [64, 256, 13, 13]               0
        MaxPool2d-13            [64, 256, 6, 6]               0
           Linear-14                 [6

In [58]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data,target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        output = model(data)
        loss = criterion(output,target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (batch_idx+1)%30 == 0:
            print(f'Train Epoch: {epoch} {100*(batch_idx+1)*batch_size/len(train_loader.dataset):.6f}%')

In [60]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output,target,reduction='sum').item()
            pred = output.max(1)[1]
            correct += pred.eq(target).sum().item()
        
        test_loss /= len(test_loader.dataset)
    
        print(f"\nTest set: Average Loss:{test_loss:.4f}, Accuracy: {100 * correct / len(test_loader.dataset)}%")
        print('='*50)

In [61]:
for epoch in range(1,num_epochs):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [3.200000]
Train Epoch: 1 [6.400000]
Train Epoch: 1 [9.600000]
Train Epoch: 1 [12.800000]
Train Epoch: 1 [16.000000]
Train Epoch: 1 [19.200000]
Train Epoch: 1 [22.400000]
Train Epoch: 1 [25.600000]
Train Epoch: 1 [28.800000]
Train Epoch: 1 [32.000000]
Train Epoch: 1 [35.200000]
Train Epoch: 1 [38.400000]
Train Epoch: 1 [41.600000]
Train Epoch: 1 [44.800000]
Train Epoch: 1 [48.000000]
Train Epoch: 1 [51.200000]
Train Epoch: 1 [54.400000]
Train Epoch: 1 [57.600000]
Train Epoch: 1 [60.800000]
Train Epoch: 1 [64.000000]
Train Epoch: 1 [67.200000]
Train Epoch: 1 [70.400000]
Train Epoch: 1 [73.600000]
Train Epoch: 1 [76.800000]
Train Epoch: 1 [80.000000]
Train Epoch: 1 [83.200000]
Train Epoch: 1 [86.400000]
Train Epoch: 1 [89.600000]
Train Epoch: 1 [92.800000]
Train Epoch: 1 [96.000000]
Train Epoch: 1 [99.200000]

Test set: Average Loss:0.3800, Accuracy: 85.98%
Train Epoch: 2 [3.200000]
Train Epoch: 2 [6.400000]
Train Epoch: 2 [9.600000]
Train Epoch: 2 [12.800000]
Train Epoch: